### Load libraries, functions, palette, theme

In [1]:
import os
from pathlib import Path

In [2]:
path_current = os.getcwd()
path_main = str(Path(path_current).parent)

In [3]:
# change directory to main path
os.chdir(path_main)

In [4]:
# load functions
from _functions import *

In [5]:
# load libraries and palette
%run _libraries.ipynb

In [6]:
# load regions data
path_regions_data = path_main + '/files/03-Regions-data'
path_change(path_regions_data)

%run _regions_data.ipynb

In [7]:
# back to current path
os.chdir(path_current)

In [8]:
# start stopwatch
start = stopwatch_start()

### Variables

In [9]:
dir = '007.1-007.3_SOL-Households'

In [10]:
dir_load_files = path_main + 'files/01-Datasets/'

In [11]:
dir_save_data = path_main + 'files/01-Datasets/'

In [12]:
dir_save_excel = path_main + 'files/02-Excel-data/'

### Load Data

In [13]:
economics_data = loadit('economics_data', dir=dir_load_files)

File 'economics_data' not found


# <font color='#2C8B6D'>I. Население</font>

# Section III. Уровень жизни

## 7. Доходы, расходы и условия проживания домашних хозяйств

### 7.1. Доходы и расходы домашних хозяйств на потребление, уровень и структура располагаемых ресурсов домашних хозяйств различных социально-экономических категорий

*Источник: https://rosstat.gov.ru/folder/13397*  
*Единицы измерения: рубли*

*Примечания:*  

In [14]:
income_link = path_main + \
    ('/files/04-Population/02-Standard-of-living/05-Households/'
     'Doh07_01.xlsx')

In [15]:
income_raw = pd.read_excel(income_link)

In [16]:
income = income_raw[:49].copy()

In [17]:
# remove symbols in 'year' row
income.iloc[0] = [str(i)[:4] for i in income.iloc[0]]

In [18]:
# rename column
income.iloc[0, 0] = 'Тип'

In [19]:
# make header from 0 row
income = transform_make_header_from_rows(
    income, rows_index=0)

In [20]:
# remove spaces in 0 column
income.iloc[:, 0] = [str(i).strip() for i in income.iloc[:, 0]]

In [21]:
# replace
income.iloc[:, 0] = \
    [i.replace('Располагаемые ресурсы - всего', 'Всего') for i in income.iloc[:, 0]]

In [22]:
for column in income.columns:
    income[column] = income[column].apply(lambda x: to_float(x))

In [23]:
income = to_round(income, 1)

In [24]:
income['index'] = income['2003'].copy()

In [25]:
# to NaN values if length less than 10
income['index'] = \
    [i if len(str(i)) > 10 else np.NaN for i in income['index'].values.ravel()]

In [26]:
income = income.reset_index(drop=True)

In [27]:
# concat strings in 'index' and drop unuseful values
income, drop_indexes1 = transform_concat_rows_strings(income, 'index')

In [28]:
# fillna by previous notna
income = transform_fill_values_by_previous(
    data=income, kind='column', column_name='index')

In [29]:
income = income.drop(drop_indexes1, axis=0)

In [30]:
income = income.loc[income['Тип'] != 'в том числе:']

In [31]:
type_values1 = [
    'Всего', 'Денежные расходы', 'Стоимость натуральных поступлений продуктов питания',
    'Стоимость натуральных поступлений непродовольственных товаров и услуг',
    'Сумма сделанных сбережений'
]

In [32]:
j = 0
for index in income.index:
    if not pd.isna(income.loc[index, '2003']):
        new_value = type_values1[j]
        income.loc[index, 'Тип'] = new_value
        j += 1
        if j > 4:
            j = 0

In [33]:
income = income.dropna()

In [34]:
income = income.set_index('index', drop=True)
income.index.name = None

In [35]:
# convert column names to int if possible (isdigit)
income.columns = [int(i) if i.isdigit() else i for i in income.columns]
income.columns.name = None

In [36]:
income.head()

,Тип,2003,2004,2005,2006,2007,2008,2009,2010,2011,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
Все домашние хозяйства,Всего,3586.6,4446.9,5745.8,7051.6,9156.9,11955.2,12419.4,14633.2,16597.1,...,21198.5,22890.1,23084.8,24209.8,24926.5,26917.7,28995.2,29204.5,31791.2,35066.1
Все домашние хозяйства,Денежные расходы,3148.5,3924.8,5057.9,6199.5,8062.4,10335.3,10737.4,12688.0,14432.0,...,18089.9,19447.2,19293.3,20583.5,21220.9,22926.4,24673.6,24504.8,26226.7,29185.2
Все домашние хозяйства,Стоимость натуральных поступлений продуктов пи...,224.2,225.6,256.0,271.5,301.0,348.3,378.9,422.3,447.1,...,468.6,490.2,585.2,598.8,585.0,569.6,605.0,628.0,656.3,712.8
Все домашние хозяйства,Стоимость натуральных поступлений непродовольс...,66.5,70.1,41.5,49.2,66.0,87.9,94.6,104.3,138.5,...,148.7,149.5,208.9,167.8,177.2,177.9,207.1,191.7,221.8,288.2
Все домашние хозяйства,Сумма сделанных сбережений,147.4,226.4,390.4,531.4,727.5,1183.7,1208.4,1418.5,1579.5,...,2491.2,2803.2,2997.4,2859.6,2943.3,3232.9,3509.5,3880.0,4686.4,4880.0


In [37]:
income[income['Тип'] == 'Всего'].T[1:].iloc[:, -2:].head()

,"Домашние хозяйства, проживающиев городской местности","Домашние хозяйства, проживающиев сельской местности"
2003,4009.0,2423.2
2004,5016.0,2851.3
2005,6529.5,3604.7
2006,7984.9,4495.4
2007,10354.6,5871.1


##### <font color='#AF4035'> Export Dataset to Excel

In [38]:
saveit_excel(
    data=income,
    filename=dir,
    path=save_excel,
    sheet='Структура ресурсов'
)

NameError: name 'save_excel' is not defined

### 7.2. Доходы и расходы домашних хозяйств на потребление, уровень и структура расходов на потребление домашних хозяйств различных социально-экономических категорий

*Источник: https://rosstat.gov.ru/folder/13397*  
*Единицы измерения: рубли в среднем на члена семьи в месяц*

*Примечания:*  

In [ ]:
consumption_link = \
    ('data/02-Population/02-Standard-of-living/05-Households/'
     'Doh07_05.xlsx')

In [ ]:
consumption_raw = pd.read_excel(consumption_link)

In [ ]:
consumption_raw.head(10)

In [ ]:
consumption = consumption_raw[7:55].copy()

In [ ]:
# remove symbols in 'year' row
consumption.iloc[0] = [str(i)[:4] for i in consumption.iloc[0]]

In [ ]:
# rename column
consumption.iloc[0, 0] = 'Тип'

In [ ]:
# make header from 0 row
consumption = transform_make_header_from_rows(
    consumption, rows_index=7)

In [ ]:
consumption.iloc[:, 0] = [str(i).strip() for i in consumption.iloc[:, 0]]

In [ ]:
consumption.iloc[:, 0] = \
    [i.replace('Расходы на потребление - всего', 'Всего') for i in consumption.iloc[:, 0]]

In [ ]:
for column in consumption.columns:
    consumption[column] = consumption[column].apply(
        lambda x: transform_to_float(x))

In [ ]:
consumption = transform_round(consumption, 1)

In [ ]:
consumption['index'] = consumption['2003'].copy()

In [ ]:
# to NaN values if length less than 10
consumption_index_values = consumption['index'].values.ravel()
consumption['index'] = \
    [i if len(str(i)) > 10 else np.NaN for i in consumption_index_values]

In [ ]:
consumption = consumption.reset_index(drop=True)

In [ ]:
# concat strings in 'index' and drop unuseful values
consumption, drop_indexes2 = transform_concat_rows_strings(consumption, 'index')

In [ ]:
# fillna by previous notna
consumption = transform_fill_values_by_previous(
    data=consumption, kind='column', column_name='index')

In [ ]:
consumption = consumption.drop(drop_indexes2, axis=0)

In [ ]:
consumption = consumption.loc[consumption['Тип'] != 'в том числе:']

In [ ]:
type_values2 = [
    'Всего',
    'Расходы на питание',
    'Расходы на непродовольственные товары',
    'Расходы на алкогольные напитки',
    'Расходы на оплату услуг',
    'Стоимость услуг, оказанных работодателем бесплатно или по льготным ценам'
]

In [ ]:
j = 0
for index in consumption.index:
    if not pd.isna(consumption.loc[index, '2003']):
        new_value = type_values2[j]
        consumption.loc[index, 'Тип'] = new_value
        j += 1
        if j > 5:
            j = 0

In [ ]:
consumption = consumption.dropna()

In [ ]:
consumption = consumption.set_index('index', drop=True)
consumption.index.name = None

In [ ]:
# convert column names to int if possible (isdigit)
consumption.columns = [int(i) if i.isdigit() else i for i in consumption.columns]
consumption.columns.name = None

In [ ]:
consumption.head()

##### <font color='#AF4035'> Export Dataset to Excel

In [ ]:
saveit_excel(
    data=consumption,
    filename=dir,
    path=save_excel,
    sheet='Структура потребления'
)

### 7.3. Располагаемые ресурсы домашних хозяйств в зависимости от места проживания по субъектам Российской Федерации

*Источник: https://rosstat.gov.ru/folder/13397*  
*Единицы измерения: рубли*

*Примечания:*  

In [ ]:
resources_link = \
    ('data/02-Population/02-Standard-of-living/05-Households/'
     'indikator_1-4.xlsx')

In [ ]:
resources_raw = pd.read_excel(resources_link, sheet_name='2014')
resources = transform_resources(
    resources_raw, 2014, FD_partial_names_list, federal_districts_names_list, 2)

In [ ]:
for i in range(2015,2023):
    if i in [2016, 2017, 2018, 2019, 2020, 2021]:
        sl = None
    else:
        sl = 2
    df = pd.read_excel(resources_link, sheet_name=str(i))
    df = transform_resources(
        df, i, FD_partial_names_list, federal_districts_names_list, sl)
    resources = pd.concat([resources, df], axis=1)

In [ ]:
resources = resources.T.copy()

In [ ]:
resources = resources.astype(float)

In [ ]:
resources = transform_round(resources, 1)

In [ ]:
# check NaNs
resources[resources.isna().any(axis=1)]

In [ ]:
resources

In [ ]:
get_data_two_level(resources, level0=['Республика Татарстан'])

##### <font color='#AF4035'> Export Dataset to Excel

In [ ]:
saveit_excel(
    data=resources,
    filename=dir,
    path=dir_save_excel,
    sheet='Ресурсы по регионам'
)

### Save Dataset

In [ ]:
economics_data['7.1'] = income

In [ ]:
economics_data['7.2'] = consumption

In [ ]:
economics_data['7.3'] = resources

In [ ]:
saveit(
    file=economics_data,
    name='economics_data',
    dir=dir_save_data
)

### Save Session

In [ ]:
save_session(dir)

### Execution time

In [ ]:
print(f'Execution time: {stopwatch.stop(start)}')